Titre....

In [ ]:
#Cellule pour lire stop_times.txt stocké sur Google Drive car trop volumineux pour être stocké sur le dépôt GitHub
import os, gdown

#lien : https://drive.google.com/file/d/16YSHTlDf0i1P3tqug_m4x6qikq6Wg0yv/view?usp=sharing
file_id = "16YSHTlDf0i1P3tqug_m4x6qikq6Wg0yv"
url = f"https://drive.google.com/uc?id={file_id}"
os.makedirs("data", exist_ok=True)
out_path = "data/gtfs/stop_times.txt"
gdown.download(url, out_path, quiet=False)  # gère aussi les gros fichiers >100 Mo

stop_times = pd.read_csv("data/gtfs/stop_times.txt") #heure d'arrivée de trip_id à stop_id


In [ ]:
import pandas as pd
import pyarrow as pa

logement = pd.read_parquet("data/poi/poi_logement.parquet")
logement['geometry']

0         b'\x01\x01\x00\x00\x00\x8d\xffL\xce\x9a\xc9\x0...
1         b'\x01\x01\x00\x00\x00\xf8\xea\x99\x14\xd5!\x0...
2         b'\x01\x01\x00\x00\x00\x99>\x85m\xd5\xff\x06@\...
3         b'\x01\x01\x00\x00\x00g\xb0N:6\xa7\x07@%\xd6\x...
4         b'\x01\x01\x00\x00\x00dSS7\xbc\xa5\x07@M\xd4w#...
                                ...                        
217465    b'\x01\x01\x00\x00\x00v\xd3\x1c\xbc\xf5-\x00@\...
217466    b'\x01\x01\x00\x00\x00\xef\xa3\xe3K\x8e.\x00@w...
217467    b'\x01\x01\x00\x00\x00,\xb7Ya\x9f-\x00@b\x97\x...
217468    b'\x01\x01\x00\x00\x00\xff\xc5@\xfb\xe2,\x00@D...
217469    b'\x01\x01\x00\x00\x00\x92\x07\xe7Pc-\x00@\xd5...
Name: geometry, Length: 217470, dtype: object